In [ ]:
pip install beautifulsoup4

In [ ]:
pip install requests

In [ ]:
pip install folium==0.5.0

In [ ]:
pip install geocoder

In [ ]:
pip install geopy

In [ ]:
# All libraries which are required for analysis

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

## Scraping the Wikipedia page for the table of postal codes of Canada

In [ ]:
toronto_neigh_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(toronto_neigh_data, "html.parser")
print(soup.title)
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

## The Postal Code table contents

In [ ]:

df=pd.DataFrame(table_contents)

# Dropping the rows where Borough is 'Not assigned'
df = df[df.Borough != 'Not assigned']

# Replacing the long name of the Boroughs with shorter names of Borough
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

## Neighbourhoods of Toronto

In [ ]:
lat_lon = pd.read_csv('/resources/1_Capstone/Geospatial_Coordinates.csv')
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
toronto_neigh_lot_lon_df = pd.merge(df, lat_lon, on='PostalCode')
df0 = toronto_neigh_lot_lon_df[toronto_neigh_lot_lon_df['Borough'].str.contains('Toronto', regex=False)]
df0

## Mapping the co-ordinates

In [ ]:
map_toronto = folium.Map(location=[43.6532,-79.33832],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(toronto_neigh_lot_lon_df['Latitude'],
                                         toronto_neigh_lot_lon_df['Longitude'],
                                         toronto_neigh_lot_lon_df['Borough'],
                                         toronto_neigh_lot_lon_df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# Foursquare 

In [ ]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds = ''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [ ]:
LIMIT = 100 
radius = 5000 
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [ ]:
#https://developer.foursquare.com/docs/resources/categories \ 4bf58dd8d48988d1d2941735
Food = '4bf58dd8d48988d1cb941735' #food truck #'4d4b7105d754a06374d81259' Food
df1=df0
#df1 = df1[df1['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_venues_food = getNearbyVenues(names=df1['Neighborhood'], latitudes=df1['Latitude'], longitudes=df1['Longitude'], radius=5000, categoryIds=Food)
toronto_venues_food.head()

In [ ]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [ ]:
map_toronto_food = folium.Map(location=[43.6532,-79.33832],zoom_start=10)
addToMap(toronto_venues_food, 'red', map_toronto_food)

map_toronto_food

In [ ]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [ ]:
down_town_grouped = toronto_venues_food.groupby('Neighborhood').count()
down_town_grouped

In [ ]:
# one hot encoding
down_town_onehot = pd.get_dummies(toronto_venues_food[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
down_town_onehot['Neighborhood'] = toronto_venues_food['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [down_town_onehot.columns[-1]] + list(down_town_onehot.columns[:-1])
down_town_onehot = down_town_onehot[fixed_columns]

down_town_onehot.head()

In [ ]:
down_town_grouped = down_town_onehot.groupby('Neighborhood').mean().reset_index()
down_town_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = down_town_grouped['Neighborhood']

for ind in np.arange(down_town_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(down_town_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
neighborhoods_venues_sorted

In [ ]:
# set number of clusters
kclusters = 4

down_town_grouped_clustering = down_town_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(down_town_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

down_town_merged = df1
down_town_merged = down_town_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

down_town_merged.head()

In [ ]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(down_town_merged['Latitude'], down_town_merged['Longitude'], down_town_merged['Neighborhood'], down_town_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
X1=down_town_merged.loc[down_town_merged['Cluster Labels'] == 0, down_town_merged.columns[[1]+[2] + list(range(5, down_town_merged.shape[1]))]]
X1

In [ ]:
X2=down_town_merged.loc[down_town_merged['Cluster Labels'] == 1, down_town_merged.columns[[1]+[2] + list(range(5, down_town_merged.shape[1]))]]
X2

In [ ]:
X3=down_town_merged.loc[down_town_merged['Cluster Labels'] == 2, down_town_merged.columns[[1] +[2]+ list(range(5, down_town_merged.shape[1]))]]
X3

In [ ]:
X4=down_town_merged.loc[down_town_merged['Cluster Labels'] == 3, down_town_merged.columns[[1]+[2] + list(range(5, down_town_merged.shape[1]))]]
X4

In [ ]:
Y0=down_town_merged.groupby('Borough').count()
x0=Y0['Neighborhood']
x0

In [ ]:
Y1=down_town_merged.groupby('1st Most Common Venue').count()
x1=Y1['Neighborhood']
x1

In [ ]:
Y2=down_town_merged.groupby('2nd Most Common Venue').count()
x2=Y2['Neighborhood']
x2

In [ ]:
Y3=down_town_merged.groupby('3rd Most Common Venue').count()
x3=Y3['Neighborhood']
x3

In [ ]:
Y4=down_town_merged.groupby('4th Most Common Venue').count()
x4=Y4['Neighborhood']
x4

In [ ]:
Z1=X1.groupby('Borough').count()
x1=Z1['Cluster Labels']
x1

In [ ]:
Z2=X2.groupby('Borough').count()
x2=Z2['Cluster Labels']
x2

In [ ]:
Z3=X3.groupby('Borough').count()
x3=Z3['Cluster Labels']
x3

In [ ]:
Z4=X4.groupby('Borough').count()
x4=Z4['Cluster Labels']
x4

In [ ]:
path='/resources/1_Capstone/wellbeing_toronto.csv'
toronto_population=pd.read_csv(path)
toronto_population.head()

In [ ]:
M=down_town_merged['Neighborhood'].tolist()
N=toronto_population['Neighbourhood'].tolist()

In [ ]:
for i in N:
    if i in M:
        print(i)

In [ ]:
s=1
t=[]
for i in N:
    z=i.split()
    m=len(z)
    j= z[-1]
    if any(j in word for word in M):
        t.append(j)
        print(s)
        s=s+1

In [ ]:
t

In [ ]:
s=1
t=[]
for i in N:
    if any(i in word for word in M):
        t.append(i)
        print(s)
        s=s+1

# END